# Vision Transformer

[Attention Is All You Need](https://arxiv.org/abs/1706.03762)の論文が発表されてからTransformerの勢いは自然言語処理に留まらず様々な分野で高い性能を叩き出している。画像処理分野でもTransformerの波は押し寄せており、[Vision Tranformer](https://arxiv.org/abs/2010.11929)という2021年にICLRで発表された論文ではImagenetのクラス分類においてトップクラスの性能を出せるTransformerベースのモデルを提案している。

ここではそのVision TransformerのモデルをベースにTransformerについて学びを深める。

### Aim
- Vision Transformerを通じてTransformerについて学ぶ

#### Reference
- [Vision Transformer入門](https://gihyo.jp/book/2022/978-4-297-13058-9)
    - 分かりやすくViTやViT周りの様々な論文なども紹介されている。  

todo 適宜追加していく、何を参考にしたか何が良かったかもう忘れちった

今回はpytorchを使用してスクラッチ実装をする

Transformerでは情報を*トークン*と呼ばれる特徴量ベクトル単位で扱う。例えば自然言語処理では「This is my pen.」という文は単語ごとに分けられ[^1] 、this, is, my, pen, ., の5つのトークンとして扱う。1つのトークンは適当な次元のベクトルとして表される。  
画像処理においてVision Transformerでは固定長の画像パッチに分割しその1つの画像パッチをトークンとする。
<!-- todo いい感じの図を入れる -->

どうトークンにするかは学習可能なパラメータによりモデルに行ってもらう。  
具体的には全結合層によりトークンに変換する。


[^1:]正確には必ずしも単語になるわけではない

In [ ]:
# input layer
# 画像を16x16 pixelのパッチに分けそれぞれをtransformerで扱うためのトークンへと変換する

import torch.nn as nn

class ImagePatchEmbedding(nn.Module):
    """
    input: image:torch.Tensor[n,c,h,w]
    output: embedding_vector:torch.Tensor[n,p,dim]
    
    nはバッチサイズ、chwは画像のカラーチャネル、高さ、幅であり、pはトークン数（＝パッチの個数）
    dimは埋め込みベクトルの長さ（ハイパーパラメータ）
    """
    def __init__(self,image_size=224,patch_size=16,in_channel=3,embedding_dim=768):
        """
        args:
            image_size:Union[int, tuple] 画像の高さと幅
            patch_size:int 1画像パッチのピクセル幅
            in_channel:int 入力画像のチャネル数
            embeedding_dim:int トークン（埋め込みベクトル）の長さ
        """
        super().__init__()
        image_size = self._pair(image_size) # if int -> tuple
        patch_size = self._pair(patch_size)
        self.image_size = image_size
        self.patch_size = patch_size
        self.grid_size = (image_size[0]//patch_size[0], image_size[1]//patch_size[1])
        self.num_patches = self.grid_size[0]*self.grid_size[1]

        self.proj = nn.Conv2d(in_channel,embedding_dim,patch_size,patch_size)
        #self.normalize = nn.LayerNorm(embedding_dim)
    
    def forward(self,x):
        assert h == self.image_size[0] and w == self.image_size[1], f'Input image size ({h}*{w}) doesn\'t match model ({self.image_size[0]}*{self.image_size[1]}).'

        x = self.proj(x)
        x = x.flatten(2).transpose(1,2) # (N,C,H,W) -> (B,P,C)
        #x = self.normalize(x)
        return x

    def _pair(self,x):
        """
        utils
        return a tuple if x is int 
        """
        return t if isinstance(t, tuple) else (t, t)